In [72]:
import h5py
import json
from PIL import Image
import tqdm
from tqdm import tqdm
import os



def get_bbox(mat_file, index):
    bbox = {}
    item = mat_file['digitStruct']['bbox'][index].item()
    bbox['label'] = []
    bbox['left'] = []
    bbox['top'] = []
    bbox['width'] = []
    bbox['height'] = []
    for key in ['label', 'left', 'top', 'width', 'height']:
        attr = mat_file[item][key]
        if attr.shape[0] == 1:
            bbox[key] = [int(attr[0][0])]
        else:
            bbox[key] = [int(mat_file[attr[j].item()][0][0]) for j in range(attr.shape[0])]
    return bbox


def parse_item(name, obj, bbox):
    """
    Fonction auxiliaire pour extraire les informations de bounding box d'un objet dans le fichier mat.
    """
    if isinstance(obj, h5py.Reference):
        obj = mat_file[obj]
    if obj.shape[0] == 1:
        bbox[name] = [int(obj[0])]
    else:
        bbox[name] = [int(obj[i][0]) for i in range(obj.shape[0])]

def mat_to_json(mat_file, output_file):
    """
    Fonction pour convertir un fichier mat en fichier json.
    """
    data = []
    for i in tqdm(range(len(mat_file['digitStruct']['bbox']))):
        filename = ''.join([chr(c[0]) for c in mat_file[mat_file['digitStruct']['name'][i][0]]])
        bbox = get_bbox(mat_file, i)
        entry = {'filename': filename, 'bbox': bbox}
        data.append(entry)
    with open(output_file, 'w') as f:
        json.dump(data, f)



def create_json():
    print('creating train json')
    mat_file = h5py.File('./train/digitStruct.mat', 'r')
    output_file = './train/digitStruct.json'
    mat_to_json(mat_file, output_file)
    mat_file.close()
    print('json created')
    print('creating test json')
    mat_file2 = h5py.File('./test/digitStruct.mat', 'r')
    output_file2 = './test/digitStruct.json'
    mat_to_json(mat_file2, output_file2)
    mat_file2.close()
    print('json created')
    
def json_to_yolo( parent_dir):
    # Chemin du fichier JSON créé précédemment
    json_file =  parent_dir +  '/digitStruct.json'
    # Dossier parent pour les images et les labels
   

    # Dossier pour les images
    img_dir = os.path.join(parent_dir, 'images')
    

    # Dossier pour les labels
    label_dir = os.path.join(parent_dir, 'labels')
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)

    # Fichier de chemin d'image et de label
    txt_file = parent_dir +'.txt'
    if os.path.exists(txt_file):
        os.remove(txt_file)
        

    # Ouverture du fichier JSON
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Itération sur les entrées JSON pour créer des fichiers de label pour chaque image
    with open(txt_file, 'w') as txt, tqdm(total=len(data)) as pbar:
        for entry in data:
            # Nom du fichier image
            img_name = entry['filename']
            bbox = entry['bbox']

            # Chemin du fichier image
            img_path = os.path.join(img_dir, img_name)

            # Récupération de la largeur et de la hauteur de l'image
            with Image.open(img_path) as img:
                width, height = img.size

            # Création du fichier de label pour l'image
            label_name = os.path.splitext(img_name)[0] + '.txt'
            label_path = os.path.join(label_dir, label_name)

            # Ouverture du fichier de label
            with open(label_path, 'w') as label:
                
                # Itération sur les objets dans l'image
                for i in range(len(bbox['label'])):
                   
                    # Récupération des coordonnées de la boîte englobante
                    l = bbox['label'][i]
                    x_min = bbox['left'][i]
                    y_min = bbox['top'][i]
                    x_max = x_min + bbox['width'][i]
                    y_max = y_min + bbox['height'][i]

                    # Normalisation des coordonnées
                    x_center = (x_min + x_max) / (2 * width)
                    y_center = (y_min + y_max) / (2 * height)
                    w = bbox['width'][i] / width
                    h = bbox['width'][i] / height

                    # Écriture des coordonnées normalisées dans le fichier de label
                    
                    label.write(f'{l} {x_center} {y_center} {w} {h}\n')

            # Écriture du chemin d'image et de label dans le fichier texte
            txt.write(f'{img_path}\n')
            pbar.update(1)
            
def main():
    create_json()
    
    json_to_yolo('train')
    json_to_yolo('test')

if __name__ == '__main__':
    main()


creating train json


100%|██████████| 33402/33402 [02:11<00:00, 253.17it/s]


json created
creating test json


100%|██████████| 13068/13068 [00:48<00:00, 269.16it/s]


json created


100%|██████████| 13068/13068 [00:11<00:00, 1106.71it/s]


In [71]:
parent_dir = 'test'
txt_file =  parent_dir +'.txt'
open(txt_file, 'w').close()
print(txt_file)

test.txt


100%|██████████| 33402/33402 [00:22<00:00, 1502.59it/s]
